## 2-8_103 汽車品質預測
（一）、設計說明：
1. 請使用隨機森林分類（Random Forest Classifier），撰寫程式，讀取汽車評估資料集cardata.txt，資料集中的前六個屬性為input_data，用來進行對第七個屬性（汽車可接受度）的分類，其取值範圍如下，每一列都包含由逗號分隔的單字清單，請將之解析分割。

欄位名稱	|欄位中文	|說明
 :---------| :-------------|:------------------------------------
buying	|買價	|取值範圍是vhigh、 high、 med、 low
maint	|維修保養價格	|取值範圍是vhigh、 high、 med、 low
doors	|門數	|取值範圍是2、 3、 4、 5等，5門含5門以上資料表示成5more
persons	|載人量	|取值範圍是2、 4等，超過4人資料表示成more
lug_boot	|行李箱的大小	|取值範圍是small、 med、 big
safety	|安全	|取值範圍是low、 med、 high
car	|汽車可接受度	|unacc為不可接受，acc為可接受， good為良好，vgood為非常好


2. 請用三折交叉驗證（three-fold cross-validation，將數據分三組，輪換著用其中兩組資料驗證分類器）來計算分類器的準確性。參數設定n_estimators=200, max_depth=8, random_state=7。
3. input_data = ['high', 'low', '2', 'more', 'med', 'high']（前面六個屬性），依據輸入值預測輸入類別Output class。
4. 請用與前相同之分類器進行"n_estimators"超參數最佳化，參數設定max_depth=8, random_state=7, parameter_grid=np.linspace(25, 200, 8).astype(int)，且改用cv=5。


（二）、請依序回答下列問題：
1. 請填入分類器的準確度Accuracy of the classifier為多少%（不需填入%，四捨五入計算至小數點後第二位）？
2. input_data = ['high', 'low', '2', 'more', 'med', 'high']（前面六個屬性）。請依據輸入值，填入預測類別的選項？
(A) unacc
(B) acc
(C) good
(D) vgood

3. 請填入"n_estimators"此一超參數的Training scores第一組第一筆數值（計算至小數點第四位後無條件捨去）？


In [1]:
# 導入需要用到的套件
import numpy as np
from sklearn import preprocessing  #交叉驗證
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.model_selection import validation_curve
#import matplotlib.pyplot as plt

In [3]:
# 載入汽車資料集
input_file = 'cardata.txt'
X = []
y = []
count = 0
with open(input_file, 'r') as f:
    for line in f.readlines():
        data = line[:-1].split(',')
        X.append(data)
X = np.array(X)

In [4]:
# 將字串資料轉換為數值資料
label_encoder = [] 
X_encoded = np.empty(X.shape)
for i, item in enumerate(X[0]):
    label_encoder.append(preprocessing.LabelEncoder())
    X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])

X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

In [5]:
# 建立隨機森林分類器，並進行訓練
params = {'n_estimators': 200, 'max_depth': 8, 'random_state': 7}
classifier = RandomForestClassifier(**params)
classifier.fit(X, y)

RandomForestClassifier(max_depth=8, n_estimators=200, random_state=7)

In [6]:
# 進行交叉驗證
from sklearn import model_selection
from sklearn.model_selection import validation_curve
accuracy = model_selection.cross_val_score(classifier, 
        X, y, scoring='accuracy', cv=3)
print("Accuracy of the classifier=" + str(round(100*accuracy.mean(), 2)) + "%")

Accuracy of the classifier=77.66%


In [7]:
# 進行交叉驗證
accuracy = model_selection.cross_val_score(classifier, 
        X, y, scoring='accuracy', cv=10)
print("Accuracy of the classifier=" + str(round(100*accuracy.mean(), 2)) + "%")

Accuracy of the classifier=83.23%


In [8]:
# 測試單個資料實例上的編碼
input_data = ['high', 'low', '2', 'more', 'med', 'high']

input_data_encoded = [-1] * len(input_data)
for i, item in enumerate(input_data):
     input_data_encoded[i] = int(label_encoder[i].transform([input_data[i]]))

input_data_encoded = np.array(input_data_encoded)


# Predict and print output for a particular datapoint
output_class = classifier.predict([input_data_encoded])
print("Output class=", label_encoder[-1].inverse_transform(output_class)[0])

Output class= acc


In [8]:

# 驗證曲線

classifier = RandomForestClassifier( max_depth= 8, random_state=7)

parameter_grid = np.linspace(25, 200, 8).astype(int)
train_scores, validation_scores = validation_curve(classifier, X, y, 
        "n_estimators", parameter_grid, cv=5)
print("##### VALIDATION CURVES #####")
print("\nParam: n_estimators\nTraining scores:\n", train_scores)
print("\nParam: n_estimators\nValidation scores:\n", validation_scores)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass param_name=n_estimators, param_range=[ 25  50  75 100 125 150 175 200] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


##### VALIDATION CURVES #####

Param: n_estimators
Training scores:
 [[0.97467438 0.97105644 0.97105644 0.97686189 0.96529284]
 [0.97322721 0.97322721 0.97539797 0.98698482 0.96963124]
 [0.97684515 0.97467438 0.97322721 0.98770788 0.98047722]
 [0.97612156 0.97684515 0.97467438 0.98770788 0.97903109]
 [0.97467438 0.97684515 0.97539797 0.98698482 0.98047722]
 [0.97539797 0.97539797 0.97756874 0.98553868 0.97903109]
 [0.97322721 0.97612156 0.97756874 0.98553868 0.97758496]
 [0.97322721 0.97612156 0.97539797 0.98626175 0.97541576]]

Param: n_estimators
Validation scores:
 [[0.66184971 0.78612717 0.79768786 0.8057971  0.88115942]
 [0.67919075 0.80057803 0.78612717 0.7884058  0.89855072]
 [0.65317919 0.79190751 0.78612717 0.78550725 0.90434783]
 [0.63294798 0.79479769 0.76589595 0.7826087  0.91304348]
 [0.66473988 0.77745665 0.79190751 0.79130435 0.90144928]
 [0.70520231 0.77745665 0.79479769 0.79130435 0.90434783]
 [0.65028902 0.7716763  0.79479769 0.7884058  0.90434783]
 [0.66763006 0.7716